# 1: Import essential libraries

In [1]:
import sys
sys.path.append('/home/quangminh/Documents/code/Python/kfot/notebooks')

In [2]:
%cd ..

/home/quangminh/Documents/code/Python/kfot


In [3]:
from transformers import CLIPModel,CLIPProcessor
import torch
from PIL import Image
import ot
from random import randint

# Disable warnings
import warnings
warnings.filterwarnings('ignore')

from optimal_transport.experiments.vision_language.datasets import *

from tqdm import tqdm

/home/quangminh/anaconda3/envs/OT-CLIP/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# 2: Embedding images and texts

In [11]:
# Model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

KeyboardInterrupt: 

In [ ]:
# # Define the transformations
# transform = transforms.Compose([
#     transforms.RandomHorizontalFlip(),   # Randomly flip the images horizontally for data augmentation
#     transforms.RandomCrop(32, padding=4), # Randomly crop the image with padding
#     transforms.ToTensor(),                # Convert PIL image to PyTorch Tensor
#     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)) # Normalize with mean and std
# ])

In [4]:
test_data = cifar10()

Files already downloaded and verified


In [ ]:
# Precompute one text embedding per class
class_text_embeds = {}
for class_name in tqdm(test_data.classes):
    text_input = processor(text=["A photo of " + class_name], return_tensors='pt', padding=True)
    with torch.no_grad():
        text_outputs = model.get_text_features(**text_input)  # Get the text embedding
        class_text_embeds[class_name] = text_outputs.squeeze(0)  # Save class embedding

100%|██████████| 10/10 [00:00<00:00, 15.68it/s]


In [ ]:
print(class_text_embeds)

{'airplane': tensor([ 5.7707e-02,  8.6992e-03,  3.1304e-02, -6.7422e-02, -2.7550e-01,
        -6.2694e-02,  2.4277e-01, -9.1931e-01, -4.9935e-02,  6.5703e-01,
        -5.2101e-01,  4.4065e-02,  1.4118e-01, -2.4775e-01,  1.4284e-01,
        -4.4539e-01,  2.9123e-01, -1.3457e-01, -1.1975e-01,  3.6795e-01,
         3.7946e-01,  2.7534e-01,  3.5243e-01, -6.6142e-02, -4.3480e-01,
         7.5996e-02, -3.1812e-01,  3.2307e-01,  3.0012e-03,  9.3821e-02,
         1.2796e-01,  4.3257e-01,  1.2445e-01,  3.2644e-01, -2.8315e-01,
        -4.3856e-02, -1.3413e-01, -1.3365e-01, -2.6523e-01,  2.1198e-01,
        -1.0863e-01, -3.3192e-01,  1.8205e-02,  3.0822e-01,  6.4445e-01,
         1.4678e-01,  2.3563e-01,  4.2482e-01, -3.4257e-01, -1.1518e-01,
         1.3485e-01, -5.3776e-01,  3.0328e-01, -2.6429e-01,  7.8785e-02,
        -5.1048e-02, -2.2613e-01,  1.1326e-01, -1.7886e-01,  3.6229e-01,
         2.6132e-01, -1.6961e-01, -5.5728e-02, -6.0154e-02,  2.5909e-01,
        -5.9209e-03, -5.1037e-02,  2.7

In [ ]:
n_classes = len(test_data.classes)
n_samples = len(test_data)  # total number of samples
res_img = torch.empty((n_samples, 512))  # Preallocate memory
res_text = torch.empty((n_samples, 512))
index = 0
for class_name in tqdm(test_data.classes):
    text_embeds = class_text_embeds[class_name]
    # Get batched data for the current class
    loader = test_data.get_batch_loader(class_name, batch_size=32)
    for batch in loader:
        images, labels, _ = batch
        # Preprocess images and text labels
        inputs = processor(images=images, return_tensors='pt', padding=True)
        with torch.no_grad():
            images_embeds = model.get_image_features(**inputs) # embeddings for this batch
        # Save the embeddings to the pre-allocated tensor
        batch_size = images_embeds.size(0)
        res_img[index:index + batch_size] = images_embeds
        res_text[index:index + batch_size] = text_embeds
        index += batch_size

100%|██████████| 10/10 [13:48<00:00, 82.83s/it]


In [ ]:
with open('data/cifar10/embedded/cifar10-image-embeds.pt','wb') as f:
    torch.save(res_img,f)

In [ ]:
with open('data/cifar10/embedded/cifar10-text-embeds.pt','wb') as f:
    torch.save(res_text,f)

In [5]:
# test 
img_embedding, text_embedding = torch.load('data/cifar10/embedded/cifar10-image-embeds.pt'), torch.load('data/cifar10/embedded/cifar10-text-embeds.pt')

In [6]:
print(img_embedding) 
print(text_embedding)

tensor([[ 0.3643, -0.2160, -0.0872,  ...,  0.7324, -0.4455, -0.1589],
        [ 0.3119,  0.0768,  0.0841,  ...,  0.6079,  0.0045,  0.0855],
        [-0.0894, -0.1891, -0.2045,  ...,  0.8593,  0.0405, -0.1374],
        ...,
        [ 0.1753, -0.2728, -0.1391,  ...,  0.8199,  0.1093,  0.2186],
        [ 0.2695, -0.0268, -0.5799,  ...,  0.8704, -0.1973,  0.1671],
        [-0.1570,  0.0019, -0.1564,  ...,  0.4388, -0.0404,  0.1643]])
tensor([[ 0.0577,  0.0087,  0.0313,  ...,  0.0721, -0.0207, -0.1927],
        [ 0.0577,  0.0087,  0.0313,  ...,  0.0721, -0.0207, -0.1927],
        [ 0.0577,  0.0087,  0.0313,  ...,  0.0721, -0.0207, -0.1927],
        ...,
        [ 0.0658,  0.2225, -0.0120,  ..., -0.3668, -0.1807,  0.0788],
        [ 0.0658,  0.2225, -0.0120,  ..., -0.3668, -0.1807,  0.0788],
        [ 0.0658,  0.2225, -0.0120,  ..., -0.3668, -0.1807,  0.0788]])


# 3: Test zero-shot inference

In [7]:
from sklearn.metrics import accuracy_score
from optimal_transport.adapters.info import InfoOT
from optimal_transport.adapters.emd import EMD
from optimal_transport.adapters.entropic import EntropicOT
from optimal_transport.adapters.low_rank import LrSinkhornOT
from optimal_transport.adapters.kpg.kp import KPGRLKP
from optimal_transport.adapters.factored.fot import FOT
from optimal_transport.adapters.factored.kfot import KFOT
from optimal_transport.adapters.factored.lot import LOT
import torch.nn.functional as F
import ot as OT

In [8]:
# baseline: cosine similarity
from sklearn.metrics.pairwise import cosine_similarity
P = cosine_similarity(img_embedding, text_embedding)
res = [test_data.targets[x] for x in P.argmax(1)]

print(accuracy_score(test_data.targets,res))


0.1007


In [37]:
# Fused-Gromov softmax
def fgwi_softmax(image_features, text_features, a1, a2, reg, n):
    """
    image_features: image features
    text_features: text features
    a1: regularization term for scaling the KL divergence term 1
    a2: regularization term for scaling the KL divergence term 2
    reg: regularization parameter > 0
    n: number of iterations for adjusting the structural cost matrix
    """

    # KL term 1: compute the self-similarity matrix for image features
    sigma_1 = image_features @ image_features.t()
    # KL term 2: compute the self-similarity matrix for text features
    sigma_2 = text_features @ text_features.t()

    # init cost matrix
    C = 1.0 - image_features @ text_features.t()
    # init transport plan
    P = F.softmax(-C / reg)

    for _ in tqdm(range(n)):
        C = C.cuda()
        C = C - a1 * sigma_1.cuda() @ P.cuda() @ sigma_2.cuda() * a2
        P = F.softmax(-C/ reg).cpu()
        C = C.cpu()
    return P.cpu()

transport_plan = fgwi_softmax(img_embedding, text_embedding, 1e-4,1e-4,0.01,100)
res = [test_data.targets[x] for x in P.argmax(1)]

print(accuracy_score(test_data.targets,res))

100%|██████████| 100/100 [03:07<00:00,  1.87s/it]

0.1007


In [40]:
torch.cuda.empty_cache()

In [41]:
def fgw_pgrad(image_f, text_f, a1, a2, reg, N):    
    # image f:Image features
    # text f:Text features
    # a1:Regularization term for scaling the KL divergence term 1
    # a2:Regularization term for scaling the KL divergence term 2
    # reg:Regularization term > 0
    # N:Max number of iterations for adjusting the structural cost matrix
    # KL term 1: Compute the self-similarity matrix for image features
    sigma_1 = image_f @ image_f.t()
    # KL term 2: Compute the self-similarity matrix for text features
    sigma_2 = text_f @ text_f.t()
    # Construct initial cost matrix based on the negative dot product (similarity) between image and text features
    C = 1.0 - image_f @ text_f.t()
    # Initialize the transport plan P
    P = F.softmax(-C)
    # Iteratively refine the structural cost matrix
    for i in tqdm(range(N)):
        # Adjust the cost matrix C
        C = C.cuda()
        grad = C - a1 * sigma_1.cuda() @ P.cuda() @ sigma_2.cuda() * a2
        K = P.cuda() * torch.exp(-grad/reg)
        del grad
        P = K / K.sum(dim=1,keepdim=True, dtype='cuda').cpu() # Row normalization
        del K
        C = C.cpu()
        sigma_1=sigma_1.cpu()
        sigma_2 = sigma_2.cpu()
     
        torch.cuda.empty_cache()  # Free GPU memory
        
        # Return transport plan P
    return P

P = fgw_pgrad(img_embedding,text_embedding,0.01,0.01,0.01,5)
res = [test_data.targets[x] for x in P.argmax(1)]

accuracy_score(test_data.targets,res)

  0%|          | 0/5 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 382.00 MiB. GPU 0 has a total capacity of 3.81 GiB of which 343.19 MiB is free. Including non-PyTorch memory, this process has 3.46 GiB memory in use. Of the allocated memory 3.37 GiB is allocated by PyTorch, and 11.88 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [42]:
ot = LrSinkhornOT()
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(),np.ones(img_embedding.shape[0])/img_embedding.shape[0],np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data.targets,res))

0.0971


In [7]:
ot = EntropicOT()
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(), np.ones(img_embedding.shape[0])/img_embedding.shape[0], np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data.targets,res))

0.1038


In [8]:
ot = InfoOT(num_iters=10)
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(),np.ones(img_embedding.shape[0])/img_embedding.shape[0],np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = ot.P_.argmax(axis=1)

print(accuracy_score(test_data.targets,res))

0.0


In [22]:
ot = EMD()
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(),np.ones(img_embedding.shape[0])/img_embedding.shape[0],np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = ot.P_.argmax(axis=1)

print(accuracy_score(test_data.targets,res))

TypeError: emd() got an unexpected keyword argument 'maxItermax'

In [10]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
import numpy as np

# Define the number of anchor points (keypoints)
n_anchors = 10

# Run K-Means clustering on both source (img_embedding) and target (text_embedding)
kmeans_source = KMeans(n_clusters=n_anchors, random_state=42).fit(img_embedding.numpy())
kmeans_target = KMeans(n_clusters=n_anchors, random_state=42).fit(text_embedding.numpy())

# Get the cluster centroids
source_centroids = kmeans_source.cluster_centers_
target_centroids = kmeans_target.cluster_centers_

# Compute the pairwise distances between the centroids
distances = cdist(source_centroids, target_centroids, metric='euclidean')

# Find the nearest matching centroids
matching_indices = np.argmin(distances, axis=1)

# Create the K list with matching pairs of anchors
K = [(i, matching_indices[i]) for i in range(n_anchors)]

ot = KFOT()
a = np.ones(img_embedding.shape[0]) / img_embedding.shape[0]  # source weights
b = np.ones(text_embedding.shape[0]) / text_embedding.shape[0]  # target weights
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(), a, b,
            K = [], ys = a, yt = b)

res = [test_data.targets[x] for x in ot.P_.argmax(1)]

print(accuracy_score(test_data,res))

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [12]:
ot = FOT(solver_kwargs={
            "r": 3, "reg": 0.01, "stopThr": 1e-7,
            "numItermax": 10000, "method": "sinkhorn_log"
        })
ot = ot.fit(img_embedding.numpy(),text_embedding.numpy(),np.ones(img_embedding.shape[0])/img_embedding.shape[0],np.ones(text_embedding.shape[0])/text_embedding.shape[0])
res = [test_data.targets[x] for x in ot.P_.argmax(1)]
print(accuracy_score(test_data.targets,res))

0.0933


In [8]:
batch_size_lot = 512
predictions = []

ot = LOT(distance=OT.dist, n_source_anchors=100, n_target_anchors=100)


for i in range(0, img_embedding.shape[0], batch_size_lot):
    img_batch = img_embedding[i:i+batch_size_lot].numpy()
    ot.fit(img_batch, text_embedding.numpy(),
                np.ones(img_batch.shape[0]) / img_batch.shape[0],
                np.ones(text_embedding.shape[0]) / text_embedding.shape[0])
    
    batch_res = [test_data.targets[x] for x in ot.P_.argmax(1)]
    predictions.extend(batch_res)

print(accuracy_score(test_data.targets[:len(predictions)], predictions))


MemoryError: Unable to allocate 19.5 GiB for an array with shape (1024, 10000, 512) and data type float32